### Change the resources of the queued jobs to the epyc96 partition.

In [3]:
from perqueue.queue import PersistentQueue
from perqueue.selection import Selection
from ase.db import connect
from perqueue.task_classes.util_classes import Resources

# Get the entries from the active pq database and modify their resources
with PersistentQueue() as pq:
    entries = pq.get_entries()
    
s = Selection(states='q')
targets = s.filter(entries)

# Modify the resources of the entries
new_res = Resources.from_string("168:1:xeon24el8:50h")
with PersistentQueue() as pq:
    for en in targets:
        en._task.resources = new_res
        pq.save_resources(en.key, new_res, False)
        print(en)
    
        
    # pq_key = en.key
    # #print(en)
    # with PersistentQueue() as pq:
    #     if pq.get_code(pq_key).name != 'neb.py':
    #         print(f"New resources: {new_res1} for pq key: {pq_key}")
    #         pq.save_resources(pq_key, Resources(new_res1), all=False)
    #         print(en)
        #else:
        #    print(f"New resources: {new_res2} for pq key: {pq_key}")
        #    pq.save_resources(pq_key, Resources(new_res2), all=True)
        
        #pq.resubmit(en)
    
    #break

Entry(task=Task(code=PosixPath('/home/energy/armoma/phd/hex_perovs/strained/codes/apply_strain.py'), args={'system_id': 1801, 'name': 'Ba7Nb4MoO20_p1', 'in_plane': -2.0}, resources=Resources(cores=168, processes=1, tmax=180000, nodename='xeon24el8')), entry_key=1, name='apply_strain.py', mq_id=7523651, state='q', err_txt=None, entry_group_key=1, pq_args=None, data=None)
Entry(task=Task(code=PosixPath('/home/energy/armoma/phd/hex_perovs/strained/codes/apply_strain.py'), args={'system_id': 1801, 'name': 'Ba7Nb4MoO20_p1', 'in_plane': -1.0}, resources=Resources(cores=168, processes=1, tmax=180000, nodename='xeon24el8')), entry_key=4, name='apply_strain.py', mq_id=7523652, state='q', err_txt=None, entry_group_key=2, pq_args=None, data=None)
Entry(task=Task(code=PosixPath('/home/energy/armoma/phd/hex_perovs/strained/codes/apply_strain.py'), args={'system_id': 1801, 'name': 'Ba7Nb4MoO20_p1', 'in_plane': 0.0}, resources=Resources(cores=168, processes=1, tmax=180000, nodename='xeon24el8')), ent